# ORMGP hydrological raster preparation (OWRC23)
This notebook documents the creation of a set of upscaled rasters needed for watershed characterization. Version 2023.

All data needed in the `input/` directory can be accessed [here](https://www.dropbox.com/scl/fo/bfkxkkrz940eqkdsk9cqy/AJadVHg9De-SdPWFORDCIHE?rlkey=tndynpc63rclqc8tu527cxg0d&dl=0).

## Load land-use data
The primary source of the land-use data is: *Ministry of Natural Resources and Forestry, 2019. Southern Ontario Land Resource Information System (SOLRIS) Version 3.0: Data Specifications. Science and Research Branch, April 2019.* In areas to the north where SOLRIS does not cover, *Agriculture and Agri-food Canada, 2013. Data Product Specifications (ISO 19131) rev. A. 31pp.* was used as infill. More information can be found [here](https://owrc.github.io/interpolants/interpolation/landuse.html).

The land-use data has been compiled as a 21,800x19,344 15m grid that is here upscaled to a 5450x4836, 60m grid. The following code collects the 4x4 15m sub-grids that are aggregated (by the most-present ID) to every 60m cell:

In [1]:
import numpy as np
import rasterio
import imageio

with rasterio.open("input/landuse23.tif") as f: r = f.read()[0]
print(r.shape)
s = np.array(r, np.uint8).reshape(4836, 4, -1, 4).swapaxes(1,2).reshape(-1, 4*4)
print(s.shape)

(19344, 21800)
(26356200, 16)


Next, cross-referencing is accomplished to i) gather all unique SOLRIS IDs and ii) create an indexed grid referencing these unique IDs:

In [2]:
u, ix = np.unique(s, return_inverse=True)
ix = ix.reshape(s.shape)
print(u)

[ 11  21  23  41  43  52  53  64  65  81  82  83  90  91  92  93 131 135
 140 150 160 170 191 192 193 201 202 203 204 205 250 255]


The following line up-scales the SOLRIS IDs by assigning the dominant SOLRIS ID to every 60m cell, and saves the data:

In [3]:
s60 = u[np.argmax(np.apply_along_axis(np.bincount, 1, ix.reshape(-1, 4*4 ), None, np.max(ix) + 1), axis=1)].reshape(4836,5450)
print(s60.shape)
s60.tofile("output/landuse23_60.bil")
imageio.imwrite('output/landuse23_60.png', s60)

(4836, 5450)


### Compute aggregates
Next, up-scaling of land use properties (imperviousness, canopy cover, etc.) is computed by averaging the (16-cell sub-grid) 15m values over the 60m cell.

#### load lookup table:

In [4]:
import pandas as pd

df = pd.read_csv("input/lookup_200731.csv").fillna(0.0)
df['PerOW'] = np.select([df['Value'] == 170], [1.0], 0.0)
df['PerWL'] = np.select([(df['Value'] >= 131) & (df['Value'] <= 160)], [1.0], 0.0)

#### fraction of impervious cover:

In [5]:
# dimp = {i:0. for i in u}
# dimp[201] = .85
# dimp[202] = .1
# dimp[203] = .90
dimp = pd.Series(df.PerImp.values,index=df.Value).to_dict()
dimp[255] = 0.
# print(dimp)

perimp = np.mean(np.vectorize(dimp.get)(s), 1).reshape(s60.shape)
perimp.astype(np.float32).tofile("output/landuse23_60_perimp.bil")
imageio.imwrite('output/landuse23_60_perimp.png', (perimp*255).astype(np.uint8))

#### fraction of canopy cover:

In [6]:
dcov = pd.Series(df.PerCov.values,index=df.Value).to_dict()
dcov[255] = 0.

percov = np.mean(np.vectorize(dcov.get)(s), 1).reshape(s60.shape)
percov.astype(np.float32).tofile("output/landuse23_60_percov.bil")
imageio.imwrite('output/landuse23_60_percov.png', (percov*255).astype(np.uint8))

#### fraction water body cover:

In [7]:
dow = pd.Series(df.PerOW.values,index=df.Value).to_dict()
dow[255] = 0.

perow = np.mean(np.vectorize(dow.get)(s), 1).reshape(s60.shape)
perow.astype(np.float32).tofile("output/landuse23_60_perow.bil")
imageio.imwrite('output/landuse23_60_perow.png', (perow*255).astype(np.uint8))

#### fraction wetland cover:

In [8]:
dwl = pd.Series(df.PerWL.values,index=df.Value).to_dict()
dwl[255] = 0.

perwl = np.mean(np.vectorize(dwl.get)(s), 1).reshape(s60.shape)
perwl.astype(np.float32).tofile("output/landuse23_60_perwl.bil")
imageio.imwrite('output/landuse23_60_perwl.png', (perwl*255).astype(np.uint8))

## Load surficial geology data
The source of the surficial geology data is: *Ontario Geological Survey 2010. Surficial geology of southern Ontario; Ontario Geological Survey, Miscellaneous Release— Data 128 – Revised.* In some areas, an infilling procedure was applied where data were unavailable; for more information, see [here](https://owrc.github.io/interpolants/interpolation/surfgeo.html).

The raster loaded below has been build using QGIS, no further processing required here. The raster represents a [1,8] index referring to the "permeability" attribute (OGS, 2010).

In [9]:
with rasterio.open("input/surfgeo23.tif") as f: r = f.read()[0]
print(r.shape)
g = np.array(r, np.uint8).reshape(4836, 4, -1, 4).swapaxes(1,2).reshape(-1, 4*4)
print(g.shape)

(19344, 21800)
(26356200, 16)


Cross-referencing:

In [10]:
u, ix = np.unique(g, return_inverse=True)
ix = ix.reshape(g.shape)
print(u)

[  0  10  20  21  22  30  40  41  50  51  52  53  54  55  60  61  62  70
  71  72  80  81  82  90  91  92  93 120 130 142 143 170 190 200 210]


Up-scales the surficial geology IDs by assigning the dominant ID to every 60m cell, and saves the data:

In [11]:
g60 = u[np.argmax(np.apply_along_axis(np.bincount, 1, ix.reshape(-1, 4*4 ), None, np.max(ix) + 1), axis=1)].reshape(4836,5450)
print(g60.shape)
g60.tofile("output/surfgeo23_60.bil")
imageio.imwrite('output/surfgeo23_60.png', g60)

(4836, 5450)


Next, the OGS permeability attributes (High, Medium-high, ... , organics, etc.) are converted to an effective hydraulic conductivity, log-scaled:

In [12]:
dperm = {0: 'Variable', 10: 'Variable', 20: 'Variable', 21: 'Variable', 22: 'Variable', 30: 'Variable', 40: 'Variable', 41: 'Variable', 
         50: 'Medium', 51: 'Medium', 52: 'Low-Medium', 53: 'Medium', 54: 'Low', 55: 'Variable', 60: 'High', 61: 'High', 62: 'High', 70: 'High', 71: 'High', 72: 'High', 
         80: 'Low', 81: 'Low', 82: 'Low', 90: 'High', 91: 'High', 92: 'High', 93: 'High', 120: 'Fluvial', 130: 'Low', 140: 'High', 142: 'High', 143: 'High', 
         170: 'Medium-High', 190: 'Fluvial', 200: 'Organic', 210: 'Variable'}
dkref = {'Low': -9., 'Low-Medium': -8., 'Medium': -7., 'Medium-High': -6., 'High': -5., 'Variable': -8., 'Fluvial': -5., 'Organic': -6.} # {1: -9., 2: -8., 3: -7., 4: -6., 5: -5., 6: -8., 7: -5., 8: -6.}
dsg = {x: dkref[y] for x,y in dperm.items()}

# geometric mean
gk = np.mean(np.vectorize(dsg.get)(g), 1).reshape(g60.shape)
gk.astype(np.float32).tofile("output/surfgeo23_60-logK.bil")
imageio.imwrite('output/surfgeo23_60-logK.png', (gk*-28).astype(np.uint8))

# # assigned to dominant class
# gk = np.vectorize(dsg.get)(g60)
# gk.astype(np.float32).tofile("output/surfgeo23_60-logK-dominant.bil")
# imageio.imwrite('output/surfgeo23_60-logK-dominant.png', (gk*-28).astype(np.uint8))

Also, create a raster of indexed permeability types:

In [13]:
diref = {'Low': 1, 'Low-Medium': 2, 'Medium': 3, 'Medium-High': 4, 'High': 5, 'Variable': 6, 'Fluvial': 7, 'Organic': 8} 
disg = {x: diref[y] for x,y in dperm.items()}

# assigned to dominant class
gisg = np.vectorize(disg.get)(g60)
gisg.astype(np.uint8).tofile("output/surfgeo23_60-iperm-dominant.bil")
imageio.imwrite('output/surfgeo23_60-iperm-dominant.png', (gisg*31).astype(np.uint8))

# Sub-watershed aggregation
The ORMGP jurisdiction has been sub-divided into 4,238 ~10km² sub-watersheds and is provided as an index raster. Using a zonal analysis, raster values aggregated above are further aggregated to every sub-watershed.

#### SWS index raster and *"indicize"*:

In [15]:
w = np.fromfile("input/PDEM-South-D2013-OWRC25-60-HC-sws10.bil", np.int32).reshape(4836,5450)
_, iw = np.unique(w, return_inverse=True)
imageio.imwrite('output/PDEM-South-D2013-OWRC25-60-HC-sws10.png', np.array(1-iw.reshape(4836,5450)/iw.max()*255, np.uint8))

#### collect zonal summations and build dataframe:

In [16]:
n = np.unique(w, return_counts=True)
sgk = np.bincount(iw, weights=gk.flatten())
spi = np.bincount(iw, weights=perimp.flatten())
spc = np.bincount(iw, weights=percov.flatten())
sow = np.bincount(iw, weights=perow.flatten())
swl = np.bincount(iw, weights=perwl.flatten())

df = pd.DataFrame(data=dict(swsid=n[0],n=n[1],perm=sgk/n[1],perimp=spi/n[1],percov=spc/n[1],perow=sow/n[1],perwl=swl/n[1]))
df = df[df.swsid != -9999]

#### Create additional fields where indices are counted per sub-watershed:

In [17]:
import json

def swscounts(a,b):
    tups = list(zip(a.flatten(),b.flatten()))
    dout = dict()
    for k,v in tups: 
        if k==-9999: continue
        if v==255: continue
        if not k in dout: dout[k]=dict()
        if not v in dout[k]: dout[k][int(v)]=0
        dout[k][v]+=1
    djson = dict()
    for k,v in dout.items():
        djson[k] = json.dumps(v)
    return djson

df['LUcnt'] = df['swsid'].map(swscounts(w,s60))
df['SGcnt'] = df['swsid'].map(swscounts(w,g60))

#### Convert "effective" permeabilities back to OGS (2010) qualitative terms:

In [18]:
dperm = {-9.: 'low', -8.: 'low-medium', -7.: 'medium', -6.: 'medium-high', -5.: 'high'}
df.perm = round(df.perm,0)
df.replace({"perm": dperm}, inplace=True)

Next, this index dataframe is merged to the sub-watershed polygon shapefile, exported as geojson.

In [20]:
import geopandas as gpd

df.to_csv("output/PDEM-South-D2013-OWRC25-60-HC-sws10-summary.csv",index=False)

gdf = gpd.read_file('input/PDEM-South-D2013-OWRC25-60-HC-sws10.geojson')
joined_gdf = gdf.merge(df, left_on='SubId', right_on="swsid")
joined_gdf.to_file('output/PDEM-South-D2013-OWRC25-60-HC-sws10.geojson', driver="GeoJSON")

print(df)

         swsid     n        perm    perimp    percov     perow     perwl  \
1       242569    78      medium  0.000000  1.000000  0.000000  0.000000   
2       248113   230      medium  0.000000  0.950951  0.036141  0.016033   
3       395236  3126      medium  0.000000  0.905600  0.059681  0.046525   
4       422516    82  low-medium  0.000000  1.000000  0.000000  0.000000   
5       536857  3433  low-medium  0.000000  0.896200  0.080414  0.040762   
...        ...   ...         ...       ...       ...       ...       ...   
4227  25970930  4211  low-medium  0.046343  0.131015  0.089409  0.281569   
4228  26069029  1062  low-medium  0.087953  0.153990  0.091337  0.230638   
4229  26107113   951         low  0.013972  0.061126  0.000000  0.075184   
4230  26145358  1470  low-medium  0.149972  0.090351  0.115136  0.067304   
4231  26161646   456         low  0.061739  0.021519  0.006442  0.000000   

                                                  LUcnt  \
1                        {"9